In [33]:
# %pip install llama-index-llms-azure-openai
# %pip install llama-index-graph-stores-nebula
# %pip install llama-index-llms-openai
# %pip install llama-index-embeddings-azure-openai


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [34]:
# !pip install llama-index transformers pyvis networkx

In [35]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')
# , device='cuda:0'

In [36]:
# Function to parse the generated text and extract the triplets
# Rebel outputs a specific format. This code is mostly copied from the model card!

def extract_triplets(input_text):
    text = triplet_extractor.tokenizer.batch_decode([triplet_extractor(input_text, return_tensors=True, return_text=False)[0]["generated_token_ids"]])[0]

    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and subject in input_text and relation != '' and relation in input_text and object_ != '' and object_ in input_text:
        triplets.append((subject.strip(), relation.strip(), object_.strip()))

    return triplets

In [37]:
import openai
import os

# Uses openai to generate natural language responses over the paul graham esay
os.environ["OPENAI_API_KEY"] = "sk-None-Y8DoDJRGH63XFz1edTyaT3BlbkFJMtiwHXQBmx5addWF1b57"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [38]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex


documents = SimpleDirectoryReader(input_files=["./paul_graham_essay.txt"]).load_data()

In [39]:
documents[0]

Document(id_='d646b309-99d0-421c-bb3f-acad04ed1e40', embedding=None, metadata={'file_path': 'paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file_type': 'text/plain', 'file_size': 75040, 'creation_date': '2024-09-30', 'last_modified_date': '2024-03-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our s

In [40]:
# !pip install -q llama-index

In [41]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=256, chunk_overlap=10)
Settings.num_output = 256
Settings.context_window = 3900

In [42]:
# a document summary index needs both an llm and embed model
# for the constructor
index = DocumentSummaryIndex.from_documents(
    documents, embed_model=embed_model, llm=llm
)

NameError: name 'DocumentSummaryIndex' is not defined

In [44]:
from llama_index.llms.openai import OpenAI

# rebel supports up to 512 input tokens, but shorter sequences also work well
service_context = ServiceContext.from_defaults(llm=OpenAI(model_name="gpt-3.5-turbo"), chunk_size=256)

index = KnowledgeGraphIndex.from_documents(documents, kg_triplet_extract_fn=extract_triplets, service_context=service_context)

ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/

In [ ]:
response = index.as_query_engine().query("What happened to the author at Interleaf?")
print(response)

main

In [46]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir= r"C:\Users\My_Pc\Desktop\Thư Viện Pháp Luật final\Tool TVPL AI\Graph_rag\cong_thuc_tinh_thue")
documents = reader.load_data()

In [47]:
documents

[Document(id_='1efa51d2-8eb1-44f4-a09c-226f60b4e346', embedding=None, metadata={'page_label': '1', 'file_name': 'CÔNG THỨC TÍNH THUẾ THU NHẬP CÁ NHÂN MỚI NHẤT.pdf', 'file_path': 'C:\\Users\\My_Pc\\Desktop\\Thư Viện Pháp Luật final\\Tool TVPL AI\\Graph_rag\\cong_thuc_tinh_thue\\CÔNG THỨC TÍNH THUẾ THU NHẬP CÁ NHÂN MỚI NHẤT.pdf', 'file_type': 'application/pdf', 'file_size': 194162, 'creation_date': '2024-10-01', 'last_modified_date': '2024-09-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='CÔNG TH ỨC TÍNH THU Ế THU NH ẬP CÁ NHÂN  MỚI NH ẤT \n1. Mức thu ế suất thu ế thu nh ập cá nhân  \nViệc xác nh ận mức đóng thu ế thu nh ập cá nhân đư ợc chia làm 2 d ạng là  biểu thu ế \nlũy ti ến từng ph ần và bi ểu thu ế toàn ph ần. \nBiểu thu ế lũy t ừng ph ần là